In [36]:
import scipy.io as sio  
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

import dan_utils

# Load Data

In [37]:
randseed = 25
dan_utils.setup_seed(randseed)
res = 11

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
id_402 = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)
part1 = pd.read_csv('../res/%i_res%i_det_partition_results1.csv'%(randseed, res))
part2 = pd.read_csv('../res/%i_res%i_det_partition_results2.csv'%(randseed, res))

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = id_402[id_402['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = id_402['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 30

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = dan_utils.get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp[:])
    v_class_temp = v_class_temp.loc[v_class_temp['id'].isin(det_list_class_temp[:])]
    v_class[i] = v_class_temp



selected_dets = pd.read_csv('../network_classification/selected_dets.csv', index_col=0)

# filt, so that only selected dets remain
class_set = [2, 3, 4]
for i in range(len(class_set)):
    cls_ = class_set[i]
    det_set = selected_dets.loc[selected_dets['class']==cls_, 'det'].values
    v_class[cls_] = v_class[cls_].loc[v_class[cls_]['id'].isin(det_set)]

# make near_road matrix
near_road_set = []
for i in range(num_class):
    det_set = v_class[i]['id'].values
#     ipdb.set_trace()
    near_road_set.append(dan_utils.rds_mat(dist_mat, det_set, seg))

There are 5 class(es)


C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
V2 = v_class[4].T

In [39]:
sio.savemat('./RF/V.mat', {'V2': V2})

In [41]:
data = []
data = sio.loadmat('./RF/RF_2.mat')
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']
y_train = np.reshape(y_train, [np.shape(y_train)[0]])
y_test = np.reshape(y_test, [np.shape(y_test)[0]])
print('input done')

mdl = RandomForestRegressor(n_estimators=50, max_depth=2, max_features='sqrt')
print('parameter done')

mdl.fit(x_train,y_train)
y_pre = mdl.predict(x_test)

sio.savemat('result1.mat',{'pre': y_pre,'true': y_test})

input done
parameter done
